In [ ]:
import argparse
import hashlib
import json
import logging
import math
import multiprocessing
import os

import numpy as np
import yaml
from egret.data.model_data import ModelData
from egret.models.acopf import create_psv_acopf_model
from pyomo.environ import Constraint, Var, minimize, value
from pyomo.opt import SolverFactory

from config import (SP_RESULTS_DIR, RO_RESULTS_DIR,
                    SPECS_COMPLETE_YAMLS, ACPF_RESULTS_DIR)
from model import DC, LPAC, Solution
from util import in_notebook, PyomoSolution

In [ ]:
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

In [ ]:
def hashfilename(tup):
    return hashlib.md5(str(tup).encode()).hexdigest()

In [ ]:
def update_md_inplace_dc(md, omega, sol, specs):

    for n in specs['N']:
        bus_dict[n].update({
            'vm': 1.0,
            'va': sol['theta'][n, omega] * 180 / math.pi
        })

    for n in specs['N']:
        for d in specs['D_n'].get(n, list()):
            load_dict[d].update({
                'in_service': True,
                'p_load_shed': specs['p_load_hi'][d] * (1 - sol['z'][d, omega]) * md.data['system']['baseMVA'],
                'q_load_shed': specs['q_load_hi'][d] * (1 - sol['z'][d, omega]) * md.data['system']['baseMVA'],
            })

    for n in specs['N']:
        for g in specs['G_n'].get(n, list()):
            generator_dict[g].update({
                'in_service': bool(sol['alpha'][n, omega]),
                'pg': sol['p_hat'][g, omega] * md.data['system']['baseMVA'],
                'qg': 0.0,
                'p_over_generation': sol['p_check'][g, omega] * md.data['system']['baseMVA'],
                'q_over_generation': 0.0,
            })

    for (n, m) in specs['E']:
        for l in specs['L_nm'][n, m]:
            branch_dict[l].update({
                'in_service': bool(sol['beta'][n, m, omega]),
                'pf': sol['p_tilde'][l, omega] * md.data['system']['baseMVA'],
                'qf': 0.0,
                'pt': -sol['p_tilde'][l, omega] * md.data['system']['baseMVA'],
                'qt': 0.0,
            })

In [ ]:
def update_md_inplace_lpac(md, omega, sol, specs):
    
    for n in specs['N']:
        bus_dict[n].update({
            'vm': specs['v'][n] + sol['phi'][n, omega],
            'va': sol['theta'][n, omega] * 180 / math.pi,
        })

    for n in specs['N']:
        for d in specs['D_n'].get(n, list()):
            load_dict[d].update({
                'in_service': True,
                'p_load_shed': specs['p_load_hi'][d] * (1 - sol['z'][d, omega]) * md.data['system']['baseMVA'],
                'q_load_shed': specs['q_load_hi'][d] * (1 - sol['z'][d, omega]) * md.data['system']['baseMVA'],
            })

    for n in specs['N']:
        for g in specs['G_n'].get(n, list()):
            generator_dict[g].update({
                'in_service': bool(sol['alpha'][n, omega]),
                'pg': sol['p_hat'][g, omega] * md.data['system']['baseMVA'],
                'qg': sol['q_hat'][g, omega] * md.data['system']['baseMVA'],
                'p_over_generation': sol['p_check'][g, omega] * md.data['system']['baseMVA'],
                'q_over_generation': 0 * md.data['system']['baseMVA'],
            })

    for (n, m) in specs['E']:
        for l in specs['L_nm'][n, m]:
            branch_dict[l].update({
                'in_service': bool(sol['beta'][n, m, omega]),
                'pf': sol['p_tilde'][l, 'f', omega] * md.data['system']['baseMVA'],
                'qf': sol['q_tilde'][l, 'f', omega] * md.data['system']['baseMVA'],
                'pt': sol['p_tilde'][l, 'b', omega] * md.data['system']['baseMVA'],
                'qt': sol['q_tilde'][l, 'b', omega] * md.data['system']['baseMVA'],
            })

In [ ]:
dc_sol_keys = ['p_hat', 'p_check', 'p_tilde', 'z', 'theta', 'sin_hat', 'alpha', 'beta',
               'gamma_over', 'gamma_under', 'gamma']
lpac_sol_keys = ['p_hat', 'q_hat', 'p_check', 'p_tilde', 'q_tilde',
                 'z', 'theta', 'phi', 'sin_hat', 'cos_hat', 'alpha', 'beta',
                 'gamma_over', 'gamma_under', 'gamma']

# User Inputs

In [ ]:
if in_notebook():
    f = 0
    r_hat = 3
    casestudy = 'imelda'
    pftype = 'dc'
    approach = 'stochastic'
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--f')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    parser.add_argument('--approach')
    args = parser.parse_args()
    f = int(args.f)
    r_hat = int(args.rhat)
    pftype = str(args.pftype)
    casestudy = str(args.casestudy)
    approach = str(args.approach)

In [ ]:
if approach == 'stochastic':
    MY_RESULTS_DIR = SP_RESULTS_DIR
elif approach == 'robust':
    MY_RESULTS_DIR = RO_RESULTS_DIR
else:
    raise ValueError('`approach` must be either "stochastic" or "robust"')

In [ ]:
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    specs['options']['approach'] = 'stochastic'
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]
    
# instantiate a model
if pftype in ['dc']:
    MODEL = DC(**specs)

elif pftype in ['lpacc', 'lpacf', 'qpac']:
    MODEL = LPAC(**specs)

# budget constraint
MODEL.con_resource_hi.RHS = f

# update
MODEL.update()

In [ ]:
# load the identified optimal solution
zipfile = os.path.join(MY_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
sol_part = Solution.from_zip(zipfile)

# fix identified optimal solution
for (k, r), val in sol_part['x'].round().items():
    if val > 0.5:
        MODEL.x[k, r].lb = MODEL.x[k, r].ub = 1
    else:
        MODEL.x[k, r].lb = MODEL.x[k, r].ub = 0

In [ ]:
# solve the model
MODEL.model.setParam('OutputFlag', 0)
MODEL.model.setParam('MIPGap', 0.00)
MODEL.model.setParam('Threads', np.floor(multiprocessing.cpu_count()))
MODEL.model.setParam('ImpliedCuts', 2)
MODEL.model.setParam('PreSolve', 2)
MODEL.model.setParam('CutPasses', 10)
MODEL.model.setParam('MIPFocus', 2)
MODEL.update()
MODEL.solve()

# store the object
sol_full = Solution.from_solved_instance(MODEL)

In [ ]:
md = ModelData()

md.data['system']['model_name'] = 'ACTIVSg2000reduced663'
md.data['system']['mpc-version'] = '2'
md.data['system']['baseMVA'] = 100.0
md.data['system']['reference_bus'] = specs['n_ref']
md.data['system']['reference_bus_angle'] = 0

bus_dict = dict()
load_dict = dict()
generator_dict = dict()
branch_dict = dict()

for n in specs['N']:
    bus_dict[n] = {
        'v_min': specs['v_lo'][n],
        'v_max': specs['v_hi'][n],
    }

for n in specs['N']:
    for d in specs['D_n'].get(n, list()):
        load_dict[d] = {
            'p_load': specs['p_load_hi'][d] * md.data['system']['baseMVA'],
            'q_load': specs['q_load_hi'][d] * md.data['system']['baseMVA'],
            'bus': n,
        }

for n in specs['N']:
    for g in specs['G_n'].get(n, list()):
        generator_dict[g] = {
            'p_min': specs['p_gen_lo'][g] * md.data['system']['baseMVA'],
            'p_max': specs['p_gen_hi'][g] * md.data['system']['baseMVA'],
            'q_min': specs['q_gen_lo'][g] * md.data['system']['baseMVA'],
            'q_max': specs['q_gen_hi'][g] * md.data['system']['baseMVA'],
            'p_cost': {'data_type': 'cost_curve', 'cost_curve_type': 'polynomial', 'values': {0: 0, 1: 1e6}},
            'bus': n,
        }

for (n, m) in specs['E']:
    for l in specs['L_nm'][n, m]:
        y = complex(specs['g'][l], specs['b'][l])
        z = 1 / y
        r, x = z.real, z.imag
        branch_dict[l] = {
            'from_bus': n,
            'to_bus': m,
            'resistance': r,
            'reactance': x,
            'charging_susceptance': 0,
            'branch_type': 'line',
            'rating_long_term': specs['s_flow_hi'][l] * md.data['system']['baseMVA'],
            'angle_diff_min': -specs['options']['theta_delta_max'] * 180 / math.pi,
            'angle_diff_max': specs['options']['theta_delta_max'] * 180 / math.pi,
        }

md.data['elements']['bus'] = bus_dict        
md.data['elements']['load'] = load_dict
md.data['elements']['generator'] = generator_dict
md.data['elements']['branch'] = branch_dict

md.data['system']['load_mismatch_cost'] = 0
md.data['system']['q_load_mismatch_cost'] = 0

kwargs = {'include_feasibility_slack': True}

In [ ]:
hashdict = dict()

for omega in MODEL.Omega:

    tup = tuple(sorted(sol_full['alpha'].loc[sol_full['alpha'].lt(0.5)].loc[slice(None), omega].index))
    hashtup = hashfilename(tup)
    os.makedirs(os.path.join(ACPF_RESULTS_DIR, pftype), exist_ok=True)
    sur_filename = os.path.join(ACPF_RESULTS_DIR, pftype, f'{hashtup}-{pftype}.zip')
    val_filename = os.path.join(ACPF_RESULTS_DIR, pftype, f'{hashtup}-ac.zip')

    # if the contingency has not yet been assessed under the surrogate PF model
    if not os.path.exists(sur_filename):
        sol_full_omega = Solution.from_solved_instance(MODEL)
        for key in sol_full.keys():
            if pftype in ['dc'] and key in dc_sol_keys:
                tmp = sol_full_omega[key]
                tmp = tmp.loc[tmp.index.get_level_values(-1) == omega]
                if tmp.shape[0] > 1:
                    tmp.index = tmp.index.droplevel(-1)
                    sol_full_omega._solution[key] = tmp
                else:
                    sol_full_omega._solution[key] = tmp.iloc[0]
            elif pftype in ['lpacc', 'lpacf', 'qpac'] and key in lpac_sol_keys:
                tmp = sol_full_omega[key]
                tmp = tmp.loc[tmp.index.get_level_values(-1) == omega]
                if tmp.shape[0] > 1:
                    tmp.index = tmp.index.droplevel(-1)
                    sol_full_omega._solution[key] = tmp
                else:
                    sol_full_omega._solution[key] = tmp.iloc[0]
            else:
                sol_full_omega._solution.pop(key)
        sol_full_omega.to_zip(sur_filename)

    # if the contingency has not yet been validated under the ACOPF model
    if not os.path.exists(val_filename):
        if pftype in ['dc']:
            update_md_inplace_dc(md, omega, sol_full, specs)
        elif pftype in ['lpacc', 'lpacf', 'qpac']:
            update_md_inplace_lpac(md, omega, sol_full, specs)
        acopf, _ = create_psv_acopf_model(md, **kwargs)

        acopf.del_component('obj')

        @acopf.Objective(sense=minimize)
        def obj(acopf):
            p_load_shed_tot = sum(acopf.p_load_shed[d] for d in acopf.p_load_shed)
            p_over_generation_tot = 1e3 * sum(acopf.p_over_generation[g] for g in acopf.p_over_generation)
            q_over_generation_tot = 1e3 * sum(acopf.q_over_generation[g] for g in acopf.q_over_generation)
            return p_load_shed_tot + p_over_generation_tot + q_over_generation_tot

        @acopf.Constraint(acopf._var_pl_index_set)
        def eq_shed_proportion(acopf, n):
            if acopf.pl[n].value > 0:
                return acopf.q_load_shed[n] == acopf.ql[n].value / acopf.pl[n].value * acopf.p_load_shed[n]
            else:
                return acopf.q_load_shed[n] == 0

        for n in acopf.p_load_shed:
            acopf.p_load_shed[n] = sum(md.data['elements']['load'][d]['p_load_shed']
                                       for d in specs['D_n'].get(n, []))

        solver = SolverFactory('ipopt')
        result = solver.solve(acopf, tee=True)
        PyomoSolution.from_solved_instance(acopf).to_zip(val_filename)

    hashdict[omega] = hashtup
    print(f, omega, hashtup)

os.makedirs(os.path.join(ACPF_RESULTS_DIR, 'results'), exist_ok=True)
jsonfilename = os.path.join(ACPF_RESULTS_DIR, 'results', f'{approach}-{casestudy}-{pftype}-f{f}-r{r_hat}.json')
with open(jsonfilename, 'w') as fh:
    json.dump(hashdict, fh)